# 目次

2.　離散化手法（1週目～2週目）
  - Lax-Wendroff法
  - 2次精度風上差分
  - QUICK

# 移流方程式の全体のプログラムについて

まずは、前回のプログラムを以下にコピーしておきます。

In [15]:
# 必要なライブラリの設定
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import glob

# 初期状態の設定
def initial_state(x_min, x_max, dx):
    x = np.arange(x_min, x_max, dx)     # 1次元計算格子

    # 初期分布uと輸送速度c
    u = np.zeros_like(x)
    u[int(2/dx):int(5/dx)] = 1
    
    return x, u

# グラフのプロットの設定
def plot(x, u, i, dir_):
    ''' シミュレーション結果をプロットし画像を保存する '''
    # フォントの種類とサイズを設定する。
    plt.rcParams['font.size'] = 16
    #plt.rcParams['font.family'] = 'sans-serif'
    #plt.rcParams['font.sans-serif'] = ['MS Gothic']

    # 目盛を内側にする。
    plt.rcParams['xtick.direction'] = 'in'
    plt.rcParams['ytick.direction'] = 'in'

    # グラフの上下左右に目盛線を付ける。
    fig = plt.figure()

    ax1 = fig.add_subplot(111)
    ax1.yaxis.set_ticks_position('both')
    ax1.xaxis.set_ticks_position('both')

    # 軸のラベルを設定する。
    ax1.set_xlabel('x')
    ax1.set_ylabel('u')

    # スケールの設定をする。
    ax1.set_xlim(np.min(x), np.max(x)-1)
    ax1.set_ylim(-0.2, 1.2)

    # プロットを行う。
    ax1.plot(x, u, label='Result', lw=1, color='red')

    # レイアウト設定
    fig.tight_layout()

    # 画像を保存する。
    # dirフォルダが無い時に新規作成
    save_dir = dir_
    if os.path.exists(save_dir):
        pass
    else:
        os.mkdir(save_dir)

    # 画像保存パスを準備
    path = os.path.join(*[save_dir, str("{:05}".format(i)) + '.png'])

    # 画像を保存する
    plt.savefig(path)

    # グラフを表示する。
    #plt.show()
    plt.close()
    return

def create_gif(in_dir, out_filename):
    ''' imgフォルダの複数画像からGIF画像を作る '''
    path_list = []
    path_list = sorted(glob.glob(os.path.join(*[in_dir, '*'])))  # ファイルパスをソートしてリストする
    imgs = []                                                    # 画像をappendするための空配列を定義

    # ファイルのフルパスからファイル名と拡張子を抽出
    for i in range(len(path_list)):
        img = Image.open(path_list[i])                           # 画像ファイルを1つずつ開く
        imgs.append(img)                                         # 画像をappendで配列に格納していく

    # appendした画像配列をGIFにする。durationで持続時間、loopでループ数を指定可能。
    imgs[0].save(out_filename,
                 save_all=True, append_images=imgs[1:], optimize=False, duration=20, loop=0)
    return

そしてメインのプログラムは以下とします。

```python
if __name__ == '__main__':
    ''' 条件設定を行いシミュレーションを実行、流れのGIF画像を作成する '''
    # 時間項の条件
    dt = 0.1

    # 空間項の条件
    dx = 0.1
    x_min = 0
    x_max = 10
    
    c = 1
    # クーラン数の確認
    co= c * dt / dx

    print(f'クーラン数 = {co}')

    x, u = initial_state(x_min, x_max, dx) #初期状態
    sol_1d_advection_upwind(x=x, u=u, dt=dt, dx=dx, c=c, step=40, schems='upwind_co1.0') # シミュレーションの実行
```
最後の行の関数の呼び出しを以下どれを使うかによって数値解法を切り替えることができます。

- sol_1d_advection_upwind　後退差分
- sol_1d_advection_forwardDiff  前進差分
- sol_1d_advection_FTCS  中心差分

# Lax-Wendroff法(クーラン数0.5)

続いてLax-Wendroff法についてプログラムを作りましょう。

Lax-Wendroff法は時間に対して2次精度、空間に対しても2次精度のスキームです。

時間に対してテーラー展開すると、

\begin{align*}
u^{n+1}_{j}=u^{n}_{j}-c\frac{\partial u}{\partial x}\Delta t+\frac{1}{2}c^2\frac{\partial^2 u}{\partial x^2}\Delta t^2
\end{align*}
中心差分との違いは時間の2次精度という点です。

空間ラベルiの微分$\big(\frac{\partial u}{\partial x}\big)_{j}\simeq \frac{u_{j+1}-u_{j-1}}{2\Delta x}$とすることです。

さらに時間の2次の項もあるため、空間微分の$\big(\frac{\partial^2 u}{\partial x^2}\big)_{j}\simeq \frac{u_{j+1}-2u_{j}u_{j-1}}{2\Delta x^2}$が必要になります。

ゆえにLax-Wendroff法の移流方程式の差分は以下のようになります。

\begin{align*}
u^{n+1}_{j}=u^{n}_{j}-\frac{c\Delta t}{\Delta x}\frac{u^{n}_{j+1}-u^{n}_{j-1}}{2}+\frac{1}{2}c^2\Delta t^2\frac{u_{j+1}-2u_{j}+u_{j-1}}{\Delta x^2}
\end{align*}
を解いています。

前進差分のプログラム同様、Lax-Wendroff法は関数名sol_1d_advection_LaxWendroffという名前にし、移流方程式の差分は以下に変更します。
```python
u[j] = un[j] - c * (dt / dx) * (un[j+1] - un[j-1])/2 + (c * (dt / dx))**2 * (un[j+1] -2*un[j] + un[j-1])/2
```

In [16]:
# 移流方程式の差分（陽解法、Lax-Wendrof法）
def sol_1d_advection_LaxWendroff(x, u, dt, dx, c, step, schems):
    dir_ = 'img'
    ''' 輸送速度が正の1次元移流方程式を数値計算する '''
    for i in range(step):
        if i % 10 == 0:
            print('Iteration=', i)

        un = u.copy()
        for j in range(1, len(x)-1):
            # 時間項：前進差分/空間項：Lax-Wendroff法
            u[j] = un[j] - c * (dt / dx) * (un[j+1] - un[j -1])/2 + (c * (dt / dx))**2 * (un[j+1] -2*un[j] + un[j-1])/2
        
        plot(x, u, i, dir_)   
        create_gif(dir_, f'result/002_1d_advection_{schems}.gif') # gifファイルの作成

では、Lax-Wendroff法のプログラムを実行しましょう。

以下のようにメインプログラムの中の移流方程式を解く関数をLax-Wendroff法に変更します。

In [17]:
if __name__ == '__main__':
    ''' 条件設定を行いシミュレーションを実行、流れのGIF画像を作成する '''
    # 時間項の条件
    dt = 0.1

    # 空間項の条件
    dx = 0.1
    x_min = 0
    x_max = 10
    
    c = 0.5
    # クーラン数の確認
    co= c * dt / dx

    print(f'クーラン数 = {co}')

    x, u = initial_state(x_min, x_max, dx) #初期状態
    sol_1d_advection_LaxWendroff(x=x, u=u, dt=dt, dx=dx, c=c, step=40, schems="LaxWendrof_co0.5") # シミュレーションの実行

クーラン数 = 0.5
Iteration= 0
Iteration= 10
Iteration= 20
Iteration= 30


アニメーションの結果は「result/002_1d_advection_LaxWendrof_co0.5.gif」でご確認ください。

Lax-Wendroff法は空間微分の離散化に対しては2次精度であり、空間微分だけを見ると中心差分と同じ精度でありますが、中心差分よりは安定であることがわかります。

もう一度Lax-Wendroff法の離散化を確認すると、最後の項はいわゆる拡散方程式に出てくる2階微分です。
\begin{align*}
u^{n+1}_{j}=u^{n}_{j}-\frac{c\Delta t}{\Delta x}\frac{u^{n}_{j+1}-u^{n}_{j-1}}{2}+\frac{1}{2}c^2\frac{u_{j+1}-2u_{j}+u_{j-1}}{\Delta x^2}
\end{align*}

つまり、Lax-Wendroff法は中心差分の不安定な離散化手法に安定化させるための拡散項が加わっているので、中心差分のような解の振動を抑えて安定する方向にはたらいてくれたと言えます。

# 2次精度風上差分(クーラン数0.5)

2次精度風上差分とは、jでの微分$\big(\frac{\partial u}{\partial x}\big)_{j}$を、風上側のjとj-1で表現する離散化手法です。

空間微分は$\big(\frac{\partial u}{\partial x}\big)_{j}\simeq \frac{3u_{j}-4u_{j-1}+u_{j-2}}{2\Delta x}$となります。

ゆえに、移流方程式を時間に対して1次精度の陽解法、空間に対して2次精度風上差分で離散化すると、

\begin{align*}
u^{n+1}_{j}=u^{n}_{j}-\frac{c\Delta t}{\Delta x}\frac{3u^{n}_{j}-4u^{n}_{j}+u^{n}_{j-2}}{2}
\end{align*}


前進差分のプログラム同様、2次精度風上差分は関数名sol_1d_advection_upwind2という名前にし、移流方程式の差分は以下に変更します。

```python
u[j] = un[j] - c * (dt / dx) *  (3*un[j] -4*un[j-1] + un[j-2])/2
```

In [18]:
# 移流方程式の差分（陽解法、2次精度風上差分）
def sol_1d_advection_upwind2(x, u, dt, dx, c, step, schems):
    dir_ = 'img'
    ''' 輸送速度が正の1次元移流方程式を数値計算する '''
    for i in range(step):
        if i % 10 == 0:
            print('Iteration=', i)

        un = u.copy()
        for j in range(1, len(x)-1):
            # 時間項：前進差分/空間項：2次精度風上差分
            u[j] = un[j] - c * (dt / dx) *  (3*un[j] -4*un[j-1] + un[j-2])/2
        
        plot(x, u, i, dir_)   
        create_gif(dir_, f'result/002_1d_advection_{schems}.gif') # gifファイルの作成

では、2次精度風上差分法のプログラムを実行しましょう。

以下のようにメインプログラムの中の移流方程式を解く関数を2次精度風上差分に変更します。

In [19]:
if __name__ == '__main__':
    ''' 条件設定を行いシミュレーションを実行、流れのGIF画像を作成する '''
    # 時間項の条件
    dt = 0.1

    # 空間項の条件
    dx = 0.1
    x_min = 0
    x_max = 10
    
    c = 0.5
    # クーラン数の確認
    co= c * dt / dx

    print(f'クーラン数 = {co}')

    x, u = initial_state(x_min, x_max, dx) #初期状態
    sol_1d_advection_upwind2(x=x, u=u, dt=dt, dx=dx, c=c, step=40, schems="upwind2_co0.5") # シミュレーションの実行

クーラン数 = 0.5
Iteration= 0
Iteration= 10
Iteration= 20
Iteration= 30


アニメーションの結果は「result/002_1d_advection_upwind2_co0.5.gif」でご確認ください。

2次精度風上差分は不安定な結果になりました。
1次精度の風上差分は安定だったのに（解が拡散するなどはあるが）、空間微分に対して2次の精度だと不安定な振る舞いをすることがわかります。

※プログラムの中に```un[j-2]```がある場合は注意が必要です。

forループは、

```python
for j in range(1, len(x)-1):
    # 時間項：前進差分/空間項：2次精度風上差分
    u[j] = un[j] - c * (dt / dx) *  (3*un[j] -4*un[j-1] + un[j-2])/2
```

となっているため、```i=1```を```un[j-2]```に代入した際には```u[-1]```となってしまいます。
これは配列の定義としては、定義の外にアクセスをしているように思えるかもしれません。

初期状態のプログラムを確認すると、numpy配列で```u```の範囲を定義しています。
```python
# 初期状態の設定
def initial_state(x_min, x_max, dx):
    x = np.arange(x_min, x_max, dx)     # 1次元計算格子

    # 初期分布uと輸送速度c
    u = np.zeros_like(x)
    u[int(2/dx):int(5/dx)] = 1
    
    return x, u
```

numpy配列の場合は```u[-1]```は最後から-1番目のindexにアクセスするという意味なので、```u[xmax]```と同じ値にアくせしていることになります。

以下で実際に確認してみましょう。

In [20]:
test_list = np.arange(0,5,1)

test_list

array([0, 1, 2, 3, 4])

In [21]:
test_list[-1]

4

In [22]:
test_list[4] # indexは0からはじまることに注意

4

以上のように```test_list[-1]```は```test_list[4]```と同じ値を返します。

つまり、今のプログラムは周期境界条件を自動的に課していることになります。

※周期境界とは右端と左端が同じ値になるということです。

# QUICK (Quadratic Upstream Interpolation for Convection Kinematics) (クーラン数0.5)

QUICKとは、jでの微分$\big(\frac{\partial u}{\partial x}\big)_{j}=\frac{u_{j+1/2}\,\,-u_{j-1/2}}{\Delta x}$を、j+1とj-1を使ってまずはj+1/2の中間点を表現します。

つまり、移流方程式の

\begin{align*}
u^{n+1}_{j}=u^{n}_{j}-\frac{c\Delta t}{\Delta x}(u^{n}_{j+1/2}-u^{n}_{j-1/2})
\end{align*}

となり、$u^{n}_{j+1/2}$をどのように求めるかという問題になります。

テーラー展開などを使って$u^{n}_{j+1/2}$は以下のように求まります。

\begin{align*}
u^{n}_{j+1/2}=\frac{1}{8}(3u^{n}_{j+1}+6u^{n}_{j}-u^{n}_{j-1})
\end{align*}

上式を求める際にテーラー展開の2次の項まで残して計算をしたため、QUICKは空間に対して2次精度です。

移流方程式を時間に対して1次精度の陽解法、空間に対して2次精度QUICKで離散化すると、

\begin{align*}
u^{n+1}_{j}=u^{n}_{j}-\frac{c\Delta t}{\Delta x}\frac{3u^{n}_{j+1}+3u^{n}_{j}-7u^{n}_{j-1}+u^{n}_{j-2}}{8}
\end{align*}

前進差分のプログラム同様、2次精度風上差分は関数名sol_1d_advection_2oderupwindという名前にし、移流方程式の差分は以下に変更します。

```python
u[j] = un[j] - c * (dt / dx) *  (3*un[j+1] +3*un[j] - 7*un[j-1] + un[j-2])/8
```

In [23]:
# 移流方程式の差分（陽解法、QUICK）
def sol_1d_advection_QUICK(x, u, dt, dx, c, step, schems):
    dir_ = 'img'
    ''' 輸送速度が正の1次元移流方程式を数値計算する '''
    for i in range(step):
        if i % 10 == 0:
            print('Iteration=', i)

        un = u.copy()
        for j in range(1, len(x)-1):
            # 時間項：前進差分/空間項：QUICK
            u[j] = un[j] - c * (dt / dx) *  (3*un[j+1] +3*un[j] - 7*un[j-1] + un[j-2])/8
        
        plot(x, u, i, dir_)   
        create_gif(dir_, f'result/002_1d_advection_{schems}.gif') # gifファイルの作成

では、QUICKのプログラムを実行しましょう。

以下のようにメインプログラムの中の移流方程式を解く関数をQUICKに変更します。

In [24]:
if __name__ == '__main__':
    ''' 条件設定を行いシミュレーションを実行、流れのGIF画像を作成する '''
    # 時間項の条件
    dt = 0.1

    # 空間項の条件
    dx = 0.1
    x_min = 0
    x_max = 10
    
    c = 0.5
    # クーラン数の確認
    co= c * dt / dx

    print(f'クーラン数 = {co}')

    x, u = initial_state(x_min, x_max, dx) #初期状態
    sol_1d_advection_QUICK(x=x, u=u, dt=dt, dx=dx, c=c, step=40, schems="QUICK_co0.5") # シミュレーションの実行

クーラン数 = 0.5
Iteration= 0
Iteration= 10
Iteration= 20
Iteration= 30


アニメーションの結果は「result/002_1d_advection_QUICK_co0.5.gif」です。

QUICKは空間に対して2次精度であるが、解が振動する結果となりました。

## まとめ

以上をまとめると以下のことがわかりました。

- 1次元移流方程式に対して安定な数値解を得ることができたのは、空間に対して1次精度である風上差分だけである
- 2次精度は全て不安定な数値解となる

今まで得た離散化手法を以下にまとめておきます。

時間に対して以下のように1次の項まで展開します。
\begin{align*}
u^{n+1}_{j}=u^{n}_{j}-c\Delta t\frac{\partial u}{\partial x}
\end{align*}

ここから以下のように離散化手法の分類ができます。

### 風上差分 

$u^{n+1}_{j}=u^{n}_{j}- c\Delta t \frac{u^{n}_{j}-u^{n}_{j-1}}{\Delta x}$ : 時間1次精度、空間1次精度
 
 ```u[j] = un[j] - c * (dt / dx) * (un[j] - un[j-1])```
  
### 中心差分
 
 $u^{n+1}_{j}=u^{n}_{j}- c\Delta t \frac{u^{n}_{j+1}-u^{n}_{j-1}}{2\Delta x}$ : 時間1次精度、空間2次精度
 
 ```u[j] = un[j] - c * (dt / dx) * (un[j+1] - un[j-1])/2```
  
### Lax-Wendroff法

$u^{n+1}_{j}=u^{n}_{j}-\frac{c\Delta t}{\Delta x}\frac{u^{n}_{j+1}-u^{n}_{j-1}}{2}+\frac{1}{2}c^2\Delta t^2\frac{u_{j+1}-2u_{j}+u_{j-1}}{\Delta x^2}$ : 時間2次精度、空間2次精度

```u[j] = un[j] - c * (dt / dx) * (un[j+1] - un[j-1])/2 + (c * (dt / dx))**2 * (un[j+1] -2*un[j] + un[j-1])/2```
  
### 2次精度風上差分
$u^{n+1}_{j}=u^{n}_{j}-\frac{c\Delta t}{\Delta x}(3u^{n}_{j}-4u^{n}_{j}+u^{n}_{j-2})$ : 時間1次精度、空間2次精度

```u[j] = un[j] - c * (dt / dx) *  (3*un[j] -4*un[j-1] + un[j-2])```

###  QUICK
$u^{n+1}_{j}=u^{n}_{j}-\frac{c\Delta t}{\Delta x}\frac{3u^{n}_{j+1}+3u^{n}_{j}-7u^{n}_{j-1}+u^{n}_{j-2}}{8}$ : 時間1次精度、空間2次精度

```u[j] = un[j] - c * (dt / dx) *  (3*un[j+1] +3*un[j] - 7*un[j-1] + un[j-2])/8```